In [6]:
# 1. IMPORTS & CONFIGURATION
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [7]:
# UPDATE: Using the v2 dataset
BASE_DIR = r"C:\Users\ayber\OneDrive\Masaüstü\ML"
DATA_PATH = os.path.join(BASE_DIR, "Data/processed/irb2400_ready_v2.csv") 
MODEL_DIR = os.path.join(BASE_DIR, "Models")
os.makedirs(MODEL_DIR, exist_ok=True)

print(f"Loading data from: {DATA_PATH}")
df = pd.read_csv(DATA_PATH)
print(f"Data shape: {df.shape}")
print(df.head())

Loading data from: C:\Users\ayber\OneDrive\Masaüstü\ML\Data/processed/irb2400_ready_v2.csv
Data shape: (300000, 27)
   target_x  target_y  target_z   sin_yaw  cos_yaw  sin_pitch  cos_pitch  \
0 -0.080915  -0.12175  1.896487 -0.639694  0.76863   0.797355    0.60351   
1 -0.080915  -0.12175  1.896487 -0.639694  0.76863   0.797355    0.60351   
2 -0.080915  -0.12175  1.896487 -0.639694  0.76863   0.797355    0.60351   
3 -0.080915  -0.12175  1.896487 -0.639694  0.76863   0.797355    0.60351   
4 -0.080915  -0.12175  1.896487 -0.639694  0.76863   0.797355    0.60351   

   sin_roll  cos_roll       q1_in  ...  delta_q3  delta_q4  delta_q5  \
0 -0.991601  0.129333 -128.382653  ... -2.509555  0.847978  5.185268   
1 -0.991601  0.129333 -127.385707  ...  0.842248 -2.985110  2.606958   
2 -0.991601  0.129333 -129.299386  ...  1.678766  1.856383  5.139431   
3 -0.991601  0.129333 -123.512512  ...  3.328885 -4.858682 -3.323155   
4 -0.991601  0.129333 -133.310090  ... -3.489313  0.148969 -0.85370

In [8]:
# 2. DEFINE NEW FEATURES
# We replaced raw angles with Sin/Cos pairs
feature_cols = [
    'target_x', 'target_y', 'target_z', 
    'sin_yaw', 'cos_yaw', 
    'sin_pitch', 'cos_pitch', 
    'sin_roll', 'cos_roll',
    'q1_in', 'q2_in', 'q3_in', 'q4_in', 'q5_in', 'q6_in'
]

target_cols = ['delta_q1', 'delta_q2', 'delta_q3', 'delta_q4', 'delta_q5', 'delta_q6']

print(f"Input Features: {len(feature_cols)} (Added Sin/Cos)")

Input Features: 15 (Added Sin/Cos)


In [9]:
# 3. PREPROCESSING
X = df[feature_cols].values
y = df[target_cols].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_x.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)

X_test_scaled = scaler_x.transform(X_test)
y_test_scaled = scaler_y.transform(y_test)

In [10]:
# WIDER MODEL TRAINING
# NOTE on MAE: MLPRegressor does not support 'absolute_error' loss directly.
# However, making the network wider (more neurons) helps it approximate the function better,
# which reduces MAE indirectly.

print("Initializing Wider MLP Regressor...")
model = MLPRegressor(
    hidden_layer_sizes=(512, 256, 128),  # Significantly Wider & Deeper
    activation='relu',                   # ReLU activation for non-linearity
    solver='adam',                       # Adam optimizer
    max_iter=500,                        # More iterations for convergence   
    batch_size=1024,                     # Larger batch size for stability
    learning_rate_init=0.001,            # Slightly higher LR for faster convergence
    tol=0.001,                           # Loose tolerance for faster training
    early_stopping=True,                 # Enable early stopping
    validation_fraction=0.1,             # 10% for validation
    n_iter_no_change=20,                 # Lower patience for early stopping
    random_state=2,                      # For reproducibility
    verbose=True                         # Show training progress
)

print("Starting training...")
model.fit(X_train_scaled, y_train_scaled)
print("Training complete.")

Initializing Wider MLP Regressor...
Starting training...
Iteration 1, loss = 0.49595660
Validation score: 0.046503
Iteration 2, loss = 0.43323422
Validation score: 0.189327
Iteration 3, loss = 0.39269988
Validation score: 0.237761
Iteration 4, loss = 0.36816476
Validation score: 0.281435
Iteration 5, loss = 0.34844948
Validation score: 0.318316
Iteration 6, loss = 0.33223293
Validation score: 0.340754
Iteration 7, loss = 0.31998629
Validation score: 0.367788
Iteration 8, loss = 0.31080125
Validation score: 0.384048
Iteration 9, loss = 0.30477595
Validation score: 0.395539
Iteration 10, loss = 0.29853829
Validation score: 0.402057
Iteration 11, loss = 0.29545542
Validation score: 0.411325
Iteration 12, loss = 0.29212671
Validation score: 0.419316
Iteration 13, loss = 0.28891079
Validation score: 0.420790
Iteration 14, loss = 0.28686637
Validation score: 0.428777
Iteration 15, loss = 0.28449828
Validation score: 0.429570
Iteration 16, loss = 0.28309209
Validation score: 0.431132
Iteratio

In [12]:
# 5. EVALUATION
y_pred_scaled = model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("\n" + "="*30)
print("       V2 MODEL RESULTS       ")
print("="*30)
print(f"Mean Absolute Error: {mae:.4f} degrees")
print(f"RMSE:                {rmse:.4f} degrees")
print("-" * 30)

# 6. SAVE UPDATED MODELS
joblib.dump(model, os.path.join(MODEL_DIR, "mlp_baseline_v2.pkl"))
joblib.dump(scaler_x, os.path.join(MODEL_DIR, "scaler_x_v2.pkl"))
joblib.dump(scaler_y, os.path.join(MODEL_DIR, "scaler_y_v2.pkl"))
print("Saved v2 models.")


       V2 MODEL RESULTS       
Mean Absolute Error: 1.4894 degrees
RMSE:                2.1819 degrees
------------------------------
Saved v2 models.
